In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer as CountV
from sklearn.feature_selection import VarianceThreshold
import scipy
from scipy.sparse import lil_matrix

import numpy as np
import os
# import pandas as pd
import matplotlib.pyplot as plt
import time, random
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split
import datetime
import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset
# from ember_utils import *
# from ember_model import *
# from ember_pjr_utils import *



In [3]:
raw_path = '/home/mr6564/continual_research/AZ_Data/Domain/'
years = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

all_X_tr, all_Y_tr, all_Y_tr_family = [], [], []
all_X_te, all_Y_te, all_Y_te_family = [], [], []


for year in years:
    tr_file = raw_path + year + '_Domain_AZ_Train.npz'
    te_file = raw_path + year + '_Domain_AZ_Test.npz'
    
    tr_year = np.load(tr_file, allow_pickle=True)
    te_year = np.load(te_file, allow_pickle=True)
    
    X_tr_year, Y_tr_year, Y_tr_family_year = tr_year['X_train'], tr_year['Y_train'], tr_year['Y_tr_family']
    X_te_year, Y_te_year, Y_te_family_year = te_year['X_test'], te_year['Y_test'], te_year['Y_te_family']
    
    
    all_X_tr, all_Y_tr, all_Y_tr_family = np.concatenate((all_X_tr, X_tr_year)),\
                                            np.concatenate((all_Y_tr, Y_tr_year)),\
                                            np.concatenate((all_Y_tr_family, Y_tr_family_year))
    
    all_X_te, all_Y_te, all_Y_te_family = np.concatenate((all_X_te, X_te_year)),\
                                            np.concatenate((all_Y_te, Y_te_year)),\
                                            np.concatenate((all_Y_te_family, Y_te_family_year))

#print(len(np.where(Y_tr_family_year == 'goodware')[0]), len(np.where(Y_tr_year == 0)[0]))

In [4]:

def build_vocabulary(data):
    """Build a vocabulary from a list of lists of strings."""
    vocab_set = set(word for sample in data for word in sample)
    return sorted(list(vocab_set))  # Sort for consistency

def vectorize_samples(data, vocabulary):
    """Vectorize data based on the given vocabulary."""
    vocab_index = {word: idx for idx, word in enumerate(vocabulary)}
    vectorized_data = np.zeros((len(data), len(vocabulary)), dtype=int)

    for i, sample in enumerate(data):
        for word in sample:
            if word in vocab_index:
                vectorized_data[i, vocab_index[word]] = 1

    return vectorized_data




def transform_with_training_vocab(training_vocab_list, data_samples):
    """
    Transform the data samples using the vocabulary list from the training data.
    :param training_vocab_list: List of words from the training data
    :param data_samples: List of data samples (each sample is a list of words)
    :return: Vectorized data as a NumPy array
    """
    # Convert the vocabulary list to a dictionary {word: index}
    training_vocab_dict = {word: idx for idx, word in enumerate(training_vocab_list)}

    # Create a zero matrix with dimensions: number of samples x size of vocabulary
    vectorized = np.zeros((len(data_samples), len(training_vocab_list)), dtype=int)

    for i, sample in enumerate(data_samples):
        for word in sample:
            if word in training_vocab_dict:
                vectorized[i, training_vocab_dict[word]] = 1

    return vectorized



def vectorize_samples_sparse(data, vocabulary):
    vocab_index = {word: idx for idx, word in enumerate(vocabulary)}
    vectorized_data = lil_matrix((len(data), len(vocabulary)), dtype=int)
    for i, sample in enumerate(data):
        for word in sample:
            if word in vocab_index:
                vectorized_data[i, vocab_index[word]] += 1
    return vectorized_data.tocsr()  # Convert to CSR format for efficient arithmetic and matrix vector operations



def transform_with_training_vocab_sparse(training_vocab_list, data_samples):
    training_vocab_dict = {word: idx for idx, word in enumerate(training_vocab_list)}
    # Create a sparse matrix instead of a dense numpy array
    vectorized = lil_matrix((len(data_samples), len(training_vocab_list)), dtype=int)

    for i, sample in enumerate(data_samples):
        for word in sample:
            if word in training_vocab_dict:
                vectorized[i, training_vocab_dict[word]] += 1
    return vectorized.tocsr()





# all_X_tr, all_Y_tr, all_Y_tr_family
# all_X_te, all_Y_te, all_Y_te_family


data = all_X_tr
vocabulary = build_vocabulary(data)
vectorized_data = vectorize_samples_sparse(data, vocabulary)

print("Vectorized Data shape:", vectorized_data.shape)

vectorized_test_data = transform_with_training_vocab_sparse(vocabulary, all_X_te)
print(vectorized_test_data.shape)

Vectorized Data shape: (682598, 3858791)
(75848, 3858791)


In [5]:
682598 + 75848

758446

In [ ]:
selector = VarianceThreshold(threshold=0.001)

selector.fit(vectorized_data)

X_train_selected = selector.transform(vectorized_data)
X_test_selected = selector.transform(vectorized_test_data)

X_train, Y_train = X_train_selected, all_Y_tr
X_test, Y_test = X_test_selected, all_Y_te


print(f'after variance thresholding')
print(X_train_selected.shape, X_test_selected.shape)

In [ ]:
type(X_train_selected)

In [ ]:
for year in years:
    tr_file = raw_path + year + '_Domain_AZ_Train.npz'
    te_file = raw_path + year + '_Domain_AZ_Test.npz'
    
    tr_year = np.load(tr_file, allow_pickle=True)
    te_year = np.load(te_file, allow_pickle=True)
    
    X_tr_year, Y_tr_year, Y_tr_family_year = tr_year['X_train'], tr_year['Y_train'], tr_year['Y_tr_family']
    X_te_year, Y_te_year, Y_te_family_year = te_year['X_test'], te_year['Y_test'], te_year['Y_te_family']
    
    X_tr_year_vocab_vector = transform_with_training_vocab_sparse(vocabulary, X_tr_year)
    X_te_year_vocab_vector = transform_with_training_vocab_sparse(vocabulary, X_te_year)
    
    X_tr_year_transformed = selector.transform(X_tr_year_vocab_vector)
    X_te_year_transformed = selector.transform(X_te_year_vocab_vector)
    
    print(f'X_tr_year_vocab_vector {X_tr_year_vocab_vector.shape}\n X_tr_year_transformed {X_tr_year_transformed.shape}')
    print(f'X_te_year_vocab_vector {X_tr_year_vocab_vector.shape} \n X_te_year_transformed {X_tr_year_transformed.shape}')
    
    
    save_path = '/home/mr6564/continual_research/AZ_Data/Domain_Transformed/'
    tr_file_save = save_path + year + '_Domain_AZ_Train_Transformed.npz'
    te_file_save = save_path + year + '_Domain_AZ_Test_Transformed.npz'
    
    np.savez(tr_file_save, X_train=X_tr_year_transformed.toarray(), Y_train=Y_tr_year, Y_tr_family = Y_tr_family_year)
    np.savez(te_file_save, X_test=X_te_year_transformed.toarray(), Y_test=Y_te_year, Y_te_family = Y_te_family_year)
    
    

In [ ]:
tr_year = np.load(tr_file_save, allow_pickle=True)
te_year = np.load(te_file_save, allow_pickle=True)

X_tr_year, Y_tr_year_tmp, Y_tr_family_year_tmp = tr_year['X_train'], tr_year['Y_train'], tr_year['Y_tr_family']
X_te_year, Y_te_year_tmp, Y_te_family_year_tmp = te_year['X_test'], te_year['Y_test'], te_year['Y_te_family']



In [ ]:
class Ember_MLP_Net(nn.Module):
    def __init__(self, input_features):
        super(Ember_MLP_Net, self).__init__()
        
        self.fc1 = nn.Linear(input_features, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.act1 = nn.ReLU()
        self.fc1_drop = nn.Dropout(p=0.5)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc2_bn = nn.BatchNorm1d(512)
        self.act2 = nn.ReLU()
        self.fc2_drop = nn.Dropout(p=0.5)
        
        self.fc3 = nn.Linear(512, 256)
        self.fc3_bn = nn.BatchNorm1d(256)
        self.act3 = nn.ReLU()
        self.fc3_drop = nn.Dropout(p=0.5)        
        
        self.fc4 = nn.Linear(256, 128)
        self.fc4_bn = nn.BatchNorm1d(128)
        self.act4 = nn.ReLU()
        self.fc4_drop = nn.Dropout(p=0.5)  
        
        self.fc_last = nn.Linear(128, 1) 
        self.out = nn.Sigmoid()
        
        #self.activate = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc1(x)
        x = self.fc1_bn(x)
        x = self.act1(x) 
        x = self.fc1_drop(x)

        x = self.fc2(x)
        x = self.fc2_bn(x)
        x = self.act2(x) 
        x = self.fc2_drop(x)
        
        x = self.fc3(x)
        x = self.fc3_bn(x)
        x = self.act3(x) 
        x = self.fc3_drop(x)
        
        x = self.fc4(x)
        x = self.fc4_bn(x)
        x = self.act4(x)
        x = self.fc4_drop(x)
        
        x = self.fc_last(x)
        x = self.out(x)
        return x

def testing_aucscore(model, X_test, Y_test, batch_size, device):
    #X_te = torch.from_numpy(X_test).type(torch.FloatTensor)
    #y_te = torch.from_numpy(Y_test).type(torch.FloatTensor) 
    
    testloader = get_dataloader(X_test, Y_test, batch_size, train_data=False)   
    
    model.eval()
    y_pred_list = []
    y_true_list = []
    test_acc = []
    with torch.no_grad():
        for x_batch, y_batch in tqdm(testloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            tmp_test_acc = binary_acc(y_test_pred, y_batch)
            test_acc.append(tmp_test_acc.item())
            
            y_pred_tag = torch.round(y_test_pred).squeeze(1)
            y_pred_list += list(y_pred_tag.cpu().numpy())
            y_true_list += list(y_batch.cpu().numpy())
        
            
    #correct_test_results = (np.array(y_pred_list) == np.array(y_true_list)).sum()
    #acc = correct_test_results/len(y_true_list)
    
    from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, accuracy_score
    
    correct_labels, predicted_labels = np.array(y_true_list), np.array(y_pred_list)
    
    roc_auc = roc_auc_score(correct_labels, predicted_labels)
    precision = precision_score(correct_labels, predicted_labels, average='micro')
    recall = recall_score(correct_labels, predicted_labels, average='micro')
    f1score = f1_score(correct_labels, predicted_labels, average='macro')
    
    print(f'test accuracy {np.mean(test_acc)} and ROC-AUC {roc_auc}')

    #wrong_good, wrong_mal, top_k_mistaken_families = \
    #            get_mistaken_stats(np.array(y_true_list), np.array(np.round(y_pred_list)), Y_test_family, top_k)
    
    return np.mean(test_acc), roc_auc, precision, recall, f1score 

    


exp_seeds = [random.randint(1, 99999) for i in range(1)]


accs_all = []
rocauc_all = []

num_epoch = 50
batch_size = 512
patience = 5


input_features = X_train.shape[1]

replay_type, current_task = 'azdomain', 'azdomain'

X_train = X_tr_year_transformed.toarray()
X_test = X_te_year_transformed.toarray()

Y_train = Y_tr_year
Y_test = Y_te_year

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)


for exp in exp_seeds:

    start_time = time.time()
    use_cuda = True
    print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
    use_cuda = use_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    torch.manual_seed(exp)

    model = Ember_MLP_Net(input_features)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
       
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    print(f'Model has {count_parameters(model)/1000000}m parameters')    
    criterion = nn.BCELoss()    

    
#     standardization = StandardScaler()
#     standard_scaler = standardization.fit(X_train)

#     X_train = standard_scaler.transform(X_train)
#     X_test = standard_scaler.transform(X_test)
    
#     X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
#     X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)  

    
    model_save_dir = '../az_model/model/'
    create_parent_folder(model_save_dir)

    opt_save_path = '../az_model/opt/'
    create_parent_folder(opt_save_path)

    results_save_dir =  '../az_model/res/' 
    create_parent_folder(results_save_dir)

    print(f'X_train {X_train.shape} Y_train {Y_train.shape}')
    print(f'X_test {X_test.shape} Y_test {Y_test.shape}')
    
    
    task_training_time, epoch_ran, training_loss, validation_loss  = training_early_stopping(\
                                 model, model_save_dir, opt_save_path, X_train, Y_train,\
                                 X_test, Y_test, patience, batch_size, device, optimizer, num_epoch,\
                                 criterion, replay_type, current_task, exp, earlystopping=True)

    
    end_time = time.time()
    print(f'Elapsed time {(end_time - start_time)/60} mins.') 
    
    
    
    best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
    print(f'loading best model {best_model_path}')
    model.load_state_dict(torch.load(best_model_path))

    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.000001)
    best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
    print(f'loading best optimizer {best_optimizer}')
    optimizer.load_state_dict(torch.load(best_optimizer))


    acc, rocauc, precision, recall, f1score = testing_aucscore(model, X_test, Y_test, batch_size, device)
    print()
    del model_save_dir
    del opt_save_path
    del results_save_dir
    
    accs_all.append(acc)
    rocauc_all.append(rocauc)


In [ ]:
np.mean(accs_all), np.mean(rocauc_all)

In [ ]:
#with variance thresholding = 0.001
np.mean(accs_all), np.mean(rocauc_all)

# 0.001 --> 0.9644471533746527, 0.9644116198053361)
# 0.0005 --> (0.964395448745497, 0.9644228359343698)

In [ ]:


class Ember_CNN(nn.Module):
    def __init__(self, num_features):
        super(Ember_CNN, self).__init__()
        # Define the first convolutional layer
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)
        
        # Additional convolutional layers
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
        
        # Dynamically calculate the size for the fully connected layer
        self._to_linear = None
        self.convs(torch.zeros(1, 1, num_features))
        
        # Fully connected layers
        self.fc1 = nn.Linear(self._to_linear, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)

    def convs(self, x):
        # Max pooling following each convolution
        x = F.max_pool1d(F.relu(self.conv1(x)), 2)
        x = F.max_pool1d(F.relu(self.conv2(x)), 2)
        x = F.max_pool1d(F.relu(self.conv3(x)), 2)
        
        # Calculate the output size for the fully connected layer
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1]
        return x

    def forward(self, x):
        x = x.view(x.size(0), 1, -1)  # Reshape the input for Conv1d
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # Flatten the output for the fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x




num_samples, num_features = X_train.shape

# Reshape X_train to (num_samples, 1, num_features)
X_train_reshaped = X_train.reshape(num_samples, 1, num_features)

# Convert to PyTorch tensor if it's not already
X_train_tensor = torch.tensor(X_train_reshaped, dtype=torch.float32)

# Convert the NumPy array Y_train to a PyTorch tensor
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)

# Create the TensorDataset using X_train_tensor and Y_train_tensor
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)



X_test_reshaped = X_test.reshape(X_test.shape[0], 1, num_features)
X_test_tensor = torch.tensor(X_test_reshaped, dtype=torch.float32)

# Convert the NumPy array Y_train to a PyTorch tensor
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

# Create the TensorDataset using X_train_tensor and Y_train_tensor
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)


# Assuming you have training and validation datasets loaded in DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Model, optimizer, and loss function
model = Ember_CNN(num_features)  # Replace num_features with your number of features
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode

        running_loss = 0.0
        for inputs, labels in train_loader:
            # Forward pass
            outputs = model(inputs)
            outputs = outputs.squeeze()  # Squeeze the output to match target labels' shape
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation
        val_loss = 0.0
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {val_loss/len(val_loader)}")

    print("Finished Training")

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

